In [1]:
from eppy import *
from eppy.modeleditor import IDF
import os

In [31]:
from assign_params import AssignParams

In [32]:
a = AssignParams()
dp_dict = a.make_a_dict()

In [38]:
dp_dict.materials["glazing"]["u_val"]

1

In [2]:
# set the idd
iddfile = "/Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd"
IDF.setiddname(iddfile)

In [16]:
# get the idf 
idf_path = "/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fplocal_cs361/eppy_energy_models/05_18/th_0518_00h/in3.idf"

# get the weather file 
epw = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw"

# create our idf for exploring 
idf0 = IDF(idf_path, epw)

In [ ]:
# Surface Type = Ceiling, roof
# surface type = wall outside boundary condition = surface, outdoors
# st = floor


In [9]:
surface_names = ["CEILING", "ROOF", "FLOOR", "INTERIOR_WALL", "EXTERIOR_WALL"]

In [17]:
# create materials and constructions
for name in surface_names:
    # materials
    idf0.newidfobject(
    "Material:NoMass",
    Name=f"{name}_VAR_MAT",
    Roughness="Smooth",
    Thermal_Resistance=0.1,
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=0.7
    )
    # constructions 
    idf0.newidfobject(
    "Construction",
    Name=f"{name}_VAR_CONST",
    Outside_Layer=f"{name}_VAR_MAT"
    )

In [39]:
var = "CEILING"
[m for m in idf0.idfobjects["Material:NoMass"] if m.Name == f"{var}_VAR_MAT"]

[
 MATERIAL:NOMASS,
     CEILING_VAR_MAT,          !- Name
     Smooth,                   !- Roughness
     0.1,                      !- Thermal Resistance
     0.9,                      !- Thermal Absorptance
     0.7,                      !- Solar Absorptance
     0.7;                      !- Visible Absorptance]

In [27]:
s = idf0.idfobjects["BuildingSurface:Detailed"]

for surface in s:
    if surface.Surface_Type == "Floor":
        surface.Construction_Name = "FLOOR_VAR_CONST"
    if surface.Surface_Type == "Ceiling":
        surface.Construction_Name = "CEILING_VAR_CONST"
    if surface.Surface_Type == "Roof":
        surface.Construction_Name = "ROOF_VAR_CONST"
    if surface.Surface_Type == "Wall":
        if surface.Outside_Boundary_Condition == "Surface":
            surface.Construction_Name = "INTERIOR_WALL_VAR_CONST"
        else: 
            surface.Construction_Name = "EXTERIOR_WALL_VAR_CONST"

In [29]:
save_path = "/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fplocal_cs361/eppy_energy_models/05_18/th_0518_01_materials"

In [30]:
idf0.save(os.path.join(save_path, "in.idf"))